# How to do function calling using InvokeModel API and model-specific prompting 

This notebook demonstrates how we can use the `InvokeModel API` with external functions to support tool calling. 

## Overview

- **Tool calling with Anthropic Claude 3.5 Sonnet** We demonstrate how to define a single tool. In our case, for simulating a stock ticker symbol lookup tool `get_ticker_symbol` and allow the model to call this tool to return a a ticker symbol.
- **Tool calling with Meta Llama 3.3** We modify the prompts to fit Meta's suggested prompt format.

## Tool calling with Anthropic Claude 3.5 Sonnet

We set our tools and functions through Python functions.

We start by defining a tool for simulating a stock ticker symbol lookup tool (`get_ticker_symbol`). Note in our example we're just returning a constant ticker symbol for a select group of companies to illustrate the concept, but you could make it fully functional by connecting it to any stock or finance API.

In [1]:
!pip install boto3 --quiet
!pip install botocore --quiet
!pip install beautifulsoup4 --quiet
!pip install lxml --quiet

This first example leverages Claude Sonnet 3.5 in the `us-west-2` region. Later, we continue with implementations using various other models available in Amazon Bedrock. The full list of models and supported regions can be found [here](https://docs.aws.amazon.com/bedrock/latest/userguide/models-regions.html). Ensure you have access to the models discussed at the beginning of the notebook. The models are invoked via `bedrock-runtime`.

In [2]:
# Import necessary libraries
from bs4 import BeautifulSoup 
import boto3
import json


modelId = 'anthropic.claude-3-5-sonnet-20240620-v1:0'
region = 'us-west-2'

bedrock = boto3.client(
    service_name = 'bedrock-runtime',
    region_name = region,
    )

### Helper Functions & Prompt Templates

We define a few helper functions and tools that each model uses.

First, we define `ToolsList` class with a member function, namely `get_ticker_symbol`, which returns the ticker symbol of a limited set of companies. Note that there is nothing specific to the model used or Amazon Bedrock in these definitions. You can add more functions in the `ToolsList` class for added capabilities (for ex. a function that calls a finance API to retrieve stock information).

In [3]:
def get_ticker_symbol(company_name: str) -> str:

    if company_name.lower() == "general motors":
        return 'GM'
        
    elif company_name.lower() == "apple":
        return 'AAPL'

    elif company_name.lower() == "amazon":
        return 'AMZN'

    elif company_name.lower() == "3M":
        return 'MMM'

    elif company_name.lower() == "nvidia":
        return 'NVDA'

    else:
        return 'TickerNotFound'

In [4]:
"""
Additional tool functions for function calling examples with Bedrock models.
"""

def get_us_president_info(president_name: str) -> str:
    """
    Returns information about a US president based on their name.
    
    Args:
        president_name (str): The name of the US president to look up
        
    Returns:
        str: Information about the president or a message if not found
    """
    presidents_data = {
        "george washington": {
            "term": "1789-1797",
            "party": "No Party",
            "vice_president": "John Adams",
            "facts": "First president of the United States and commander-in-chief during the American Revolutionary War."
        },
        "abraham lincoln": {
            "term": "1861-1865",
            "party": "Republican",
            "vice_president": "Hannibal Hamlin, Andrew Johnson",
            "facts": "Led the United States through the Civil War and abolished slavery."
        },
        "franklin d roosevelt": {
            "term": "1933-1945",
            "party": "Democratic",
            "vice_president": "John Nance Garner, Henry A. Wallace, Harry S. Truman",
            "facts": "Only president elected to office four times. Led the US through the Great Depression and most of World War II."
        },
        "john f kennedy": {
            "term": "1961-1963",
            "party": "Democratic",
            "vice_president": "Lyndon B. Johnson",
            "facts": "Youngest elected president at age 43. Assassinated in Dallas, Texas in 1963."
        },
        "barack obama": {
            "term": "2009-2017",
            "party": "Democratic",
            "vice_president": "Joe Biden",
            "facts": "First African American president of the United States."
        },
        "joe biden": {
            "term": "2021-Present",
            "party": "Democratic",
            "vice_president": "Kamala Harris",
            "facts": "Oldest person to assume the presidency."
        }
    }
    
    # Convert to lowercase for case-insensitive matching
    president_name = president_name.lower()
    
    if president_name in presidents_data:
        data = presidents_data[president_name]
        return f"President: {president_name.title()}\nTerm: {data['term']}\nParty: {data['party']}\nVice President(s): {data['vice_president']}\nFacts: {data['facts']}"
    else:
        return "Information about this president is not available in the database."

def get_weather_data(city: str, date: str = "today") -> str:
    """
    Returns sample weather data for a given city and date.
    
    Args:
        city (str): The name of the city to get weather for
        date (str, optional): The date to get weather for. Defaults to "today".
        
    Returns:
        str: Weather information for the specified city and date
    """
    # Sample weather data for demonstration purposes
    weather_data = {
        "new york": {
            "today": {
                "temperature": "72°F (22°C)",
                "condition": "Partly Cloudy",
                "humidity": "65%",
                "wind": "10 mph NE"
            },
            "tomorrow": {
                "temperature": "75°F (24°C)",
                "condition": "Sunny",
                "humidity": "60%",
                "wind": "8 mph SW"
            }
        },
        "london": {
            "today": {
                "temperature": "62°F (17°C)",
                "condition": "Rainy",
                "humidity": "80%",
                "wind": "15 mph W"
            },
            "tomorrow": {
                "temperature": "64°F (18°C)",
                "condition": "Overcast",
                "humidity": "75%",
                "wind": "12 mph SW"
            }
        },
        "tokyo": {
            "today": {
                "temperature": "81°F (27°C)",
                "condition": "Clear",
                "humidity": "70%",
                "wind": "7 mph SE"
            },
            "tomorrow": {
                "temperature": "83°F (28°C)",
                "condition": "Sunny",
                "humidity": "65%",
                "wind": "5 mph E"
            }
        },
        "sydney": {
            "today": {
                "temperature": "68°F (20°C)",
                "condition": "Partly Cloudy",
                "humidity": "55%",
                "wind": "18 mph S"
            },
            "tomorrow": {
                "temperature": "70°F (21°C)",
                "condition": "Sunny",
                "humidity": "50%",
                "wind": "15 mph SE"
            }
        },
        "cairo": {
            "today": {
                "temperature": "95°F (35°C)",
                "condition": "Hot and Sunny",
                "humidity": "30%",
                "wind": "12 mph N"
            },
            "tomorrow": {
                "temperature": "97°F (36°C)",
                "condition": "Hot and Clear",
                "humidity": "25%",
                "wind": "10 mph NE"
            }
        }
    }
    
    # Convert to lowercase for case-insensitive matching
    city = city.lower()
    date = date.lower()
    
    if city in weather_data and date in weather_data[city]:
        data = weather_data[city][date]
        return f"Weather for {city.title()} ({date}):\nTemperature: {data['temperature']}\nCondition: {data['condition']}\nHumidity: {data['humidity']}\nWind: {data['wind']}"
    else:
        return f"Weather data for {city.title()} on {date} is not available."


The models we cover in this notebook support XML or JSON formatting to parse input prompts. We define a simple helper function converting a model's function choice into the XML format.

In [5]:
# Format the functions results for input back to the model using XML in its response
def func_results_xml(tool_name, tool_return):
   return f"""
        <function_results>
            <result>
                <tool_name>{tool_name}</tool_name>
                <stdout>
                    {tool_return}
                </stdout>
            </result>
        </function_results>"""

We define a function to parse the model's XML output into readable text. Since each model returns a different response format (i.e. Anthropic Claude's completion can be retrieved by `response['content'][0]['text']` and Meta Llama 3.1 uses `response['generation']`). Further, we create equivalent functions for the other models covered.

In [6]:
# Parses the output of Claude to extract the suggested function call and parameters
def parse_output_claude_xml(response):
    soup=BeautifulSoup(response['content'][0]['text'].replace('\n',''),"lxml")
    tool_name=soup.tool_name.string
    parameter_name=soup.parameters.contents[0].name
    parameter_value=soup.parameters.contents[0].string
    return (tool_name,{parameter_name:parameter_value})

Without `Converse`, models present some difference in their `InvokeModel API` around their hyperparameters. We define the function to invoke Anthropic models.

In [7]:
# Claude 3 invocation function
def invoke_anthopic_model(bedrock_runtime, messages, max_tokens=512,top_p=1,temp=0):

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages,
            "temperature": temp,
            "top_p": top_p,
            "stop_sequences":["</function_calls>"]
        }  
    )  
    
    response = bedrock_runtime.invoke_model(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0")
    response_body = json.loads(response.get('body').read())

    return response_body

### Creating the prompt template

We now define the system prompt provided to Claude when implementing function calling including several important components:

- An instruction describing the intent and setting the context for function calling.
- A detailed description of the tool(s) and expected parameters that Claude can suggest the use of.
- An example of the structure of the function call so that it can be parsed by the client code and ran.
- A directive to form a thought process before deciding on a function to call.
- The user query itself.

We supply `get_ticker_symbol` as a tool the model has access to respond to given type of query.

In [8]:
"""
Updated Claude system prompt that includes the new tools.
"""

system_prompt = """In this environment you have access to a set of tools you can use to answer the user's question.
    
    You may call them like this:
            
    <function_calls>
    <invoke>
    <tool_name>$TOOL_NAME</tool_name>
    <parameters>
    <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
    ...
    </parameters>
    </invoke>
    </function_calls>
            
    Here are the tools available:
    <tools>
    <tool_description>
    <tool_name>get_ticker_symbol</tool_name>
    <description>Gets the stock ticker symbol for a company searched by name. Returns str: The ticker symbol for the company stock. Raises TickerNotFound: if no matching ticker symbol is found.</description>
    <parameters>
    <parameter>
    <n>company_name</n>
    <type>string</type>
    <description>The name of the company.</description>
    </parameter>
    </parameters>
    </tool_description>
    
    <tool_description>
    <tool_name>get_us_president_info</tool_name>
    <description>Returns information about a US president based on their name.</description>
    <parameters>
    <parameter>
    <n>president_name</n>
    <type>string</type>
    <description>The name of the US president to look up.</description>
    </parameter>
    </parameters>
    </tool_description>
    
    <tool_description>
    <tool_name>get_weather_data</tool_name>
    <description>Returns sample weather data for a given city and date.</description>
    <parameters>
    <parameter>
    <n>city</n>
    <type>string</type>
    <description>The name of the city to get weather for.</description>
    </parameter>
    <parameter>
    <n>date</n>
    <type>string</type>
    <description>The date to get weather for (today or tomorrow). Optional, defaults to today.</description>
    </parameter>
    </parameters>
    </tool_description>
    </tools>
            
    Come up with a step by step plan for what steps should be taken, what functions should be called and in 
    what order. Place your thinking between <rationale> tags. Only create this rationale 1 time before 
    creating any other outputs.
            
    You will take in any outputs from called functions which will be in <fnr> tags and use 
    them to further suggests next steps and actions to take.

    If the question is unrelated to the tools available, then refuse to answer it and supply the explanation.
    """


We use the Messages API covered [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html). It manages the conversational exchanges between a user and an Anthropic Claude model (assistant). Anthropic trains Claude models to operate on alternating user and assistant conversational turns. When creating a new message, you specify the prior conversational turns with the messages parameter. The model then generates the next Message in the conversation. 

We prompt the model with a question within the scope of the tool.

In [9]:
message_list = [{"role": 'user', "content": [{"type": "text", "text": f"""
    {system_prompt}
    Here is the user's question: <question>What is the ticker symbol of General Motors?</question>

    How do you respond to the user's question?"""}]
}]

We previously added `"</function_calls>"` to the list of stop sequences letting Claude end its output prior to generating this token representing a closing bracket. Given the query, the model correctly returns its rationale and the selected tool call. Evidently, the output follows the natural language description in the system prompt passed when calling the model.

In [10]:
response = invoke_anthopic_model(bedrock, messages=message_list)
print(response['content'][0]['text'])

message_list.append({
        "role": 'assistant',
        "content": [
            {"type": "text", "text": response['content'][0]['text']}
        ]})

<rationale>
To find the ticker symbol for General Motors, I should use the `get_ticker_symbol` tool and provide the company name "General Motors" as input. This tool should return the stock ticker symbol for that company.

The steps would be:
1. Call the `get_ticker_symbol` tool with the parameter `company_name="General Motors"`
2. Use the returned ticker symbol to answer the question

Since the question is directly related to finding a company's stock ticker symbol, the available tools are sufficient to answer it.
</rationale>

<function_calls>
<invoke>
<tool_name>get_ticker_symbol</tool_name>
<parameters>
<company_name>General Motors</company_name>
</parameters>
</invoke>



In [11]:
message_list = [{"role": 'user', "content": [{"type": "text", "text": f"""
    {system_prompt}
    Here is the user's question: <question>Who was George Washington?</question>

    How do you respond to the user's question?"""}]
}]

In [12]:
response = invoke_anthopic_model(bedrock, messages=message_list)
print(response['content'][0]['text'])

message_list.append({
        "role": 'assistant',
        "content": [
            {"type": "text", "text": response['content'][0]['text']}
        ]})

<rationale>
To answer the question "Who was George Washington?", the most relevant tool available is get_us_president_info. This tool allows us to look up information about US presidents by name.

The steps would be:
1. Call the get_us_president_info tool with the parameter president_name="George Washington"
2. Use the information returned by the tool to provide details about George Washington's life and presidency.

Since the question is specifically about a US president, the other available tools (get_ticker_symbol and get_weather_data) are not relevant for answering this query.
</rationale>

<function_calls>
<invoke>
<tool_name>get_us_president_info</tool_name>
<parameters>
<president_name>George Washington</president_name>
</parameters>
</invoke>



## Tool calling with Meta Llama 3.3

Now we cover function calling using Meta Llama 3.3. 

In [13]:
# Meta Llama 3 invocation function
bedrock = boto3.client('bedrock-runtime',region_name='us-west-2')

def invoke_llama_model(bedrock_runtime, messages, max_tokens=512,top_p=1,temp=0):
    
    body=json.dumps(
        {
            "max_gen_len": max_tokens,
            "prompt": messages,
            "temperature": temp,
            "top_p": top_p,
        }  
    )  
    
    response = bedrock_runtime.invoke_model(body=body, modelId="us.meta.llama3-3-70b-instruct-v1:0")
    response_body = json.loads(response.get('body').read())

    return response_body


We define Llama's system prompt based on Meta's own [documentation](https://llama.meta.com/docs/model-cards-and-prompt-formats/llama3_1/#built-in-tooling). We define our custom tools as a JSON dictionary

In [14]:
"""
Updated Llama system prompt that includes the new tools.
"""

from datetime import datetime

system_prompt = f"""
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    Cutting Knowledge Date: December 2023
    Today Date: {datetime.today().strftime('%Y-%m-%d')}

    When you receive a tool call response, use the output to format an answer to the orginal user question.

    You are a helpful assistant with tool calling capabilities.<|eot_id|><|start_header_id|>user<|end_header_id|>

    Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.
    
    Respond in the format {{\\\"name\\\": function name, \\\"parameters\\\": dictionary of argument name and its value}}. Do not use variables.
    If the question is unrelated to the tools available, then refuse to answer it and supply the explanation.
    
    {{
        "type": "function",
        "function": {{
        "name": "get_ticker_symbol",
        "description": "Returns the ticker symbol of a company if a user searches by its company name",
        "parameters": {{
            "type": "object",
            "properties": {{
            "company_name": {{
                "type": "string",
                "description": "The name of the company."
            }}
            }},
            "required": ["company_name"]
        }}
        }}
    }},
    {{
        "type": "function",
        "function": {{
        "name": "get_us_president_info",
        "description": "Returns information about a US president based on their name",
        "parameters": {{
            "type": "object",
            "properties": {{
            "president_name": {{
                "type": "string",
                "description": "The name of the US president to look up."
            }}
            }},
            "required": ["president_name"]
        }}
        }}
    }},
    {{
        "type": "function",
        "function": {{
        "name": "get_weather_data",
        "description": "Returns sample weather data for a given city and date",
        "parameters": {{
            "type": "object",
            "properties": {{
            "city": {{
                "type": "string",
                "description": "The name of the city to get weather for."
            }},
            "date": {{
                "type": "string",
                "description": "The date to get weather for (today or tomorrow). Optional, defaults to today."
            }}
            }},
            "required": ["city"]
        }}
        }}
    }}
"""


We supply the result to the message and invoke the model to summarize the result. The model correctly summarizes the conversation flow resulting from the initial query. 

In [15]:
# Call LLama 3.3 and print response
message = f"""{system_prompt}
    Question: What is the symbol for Apple?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """

response = invoke_llama_model(bedrock, messages=message)
print(response['generation'])

 {"name": "get_ticker_symbol", "parameters": {"company_name": "Apple"}}


Once we have the necessary tool call, we can follow a similar path to other models by executing the function, then returning the result to the model.

If asking a question outside the model's scope, the model refuses to answer. It is possible to modify the instructions so the model answers the question by relying on its internal knowledge.

In [16]:
# Call LLama 3.3 and print response
message = f"""{system_prompt}
    Question: Who was George Washington?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """

response = invoke_llama_model(bedrock, messages=message)
print(response['generation'])

 {"name": "get_us_president_info", "parameters": {"president_name": "George Washington"}}


In [17]:
# Call LLama 3.3 and print response
message = f"""{system_prompt}
    Question: Whats the weather in NYC?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """

response = invoke_llama_model(bedrock, messages=message)
print(response['generation'])

 {"name": "get_weather_data", "parameters": {"city": "NYC"}}
